### Proyecto de Gobernanza y Calidad de Datos: Análisis de Permisos de Circulación

Este notebook documenta el proceso de validación, limpieza y estandarización de un dataset sintético de permisos de circulación. El objetivo es transformar datos brutos e inconsistentes en un dataset confiable, listo para su análisis o modelado predictivo.


In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset generado en el paso anterior
df = pd.read_csv('data/raw/permisos_circulacion_bruto.csv')

# Inspección inicial del dataset
print("--- Información inicial del DataFrame ---")
df.info()
print("\n--- Primeras 5 filas ---")
print(df.head())
print("\n--- Estadísticas descriptivas de columnas numéricas ---")
print(df.describe())

### 1. Validación y Detección de Inconsistencias

Antes de limpiar, es crucial identificar los problemas de calidad de los datos.

In [ ]:
# Contar valores faltantes por columna
print("--- Valores faltantes por columna ---")
print(df.isnull().sum())

# Identificar duplicados
print("\n--- Conteo de filas duplicadas ---")
print(f"Filas duplicadas: {df.duplicated().sum()}")

# Validar la columna 'duracion_dias' (valores negativos o nulos)
print("\n--- Valores no válidos en 'duracion_dias' ---")
invalid_duracion = df[df['duracion_dias'] < 0]
print(f"Registros con duración negativa: {len(invalid_duracion)}")

### 2. Limpieza y Estandarización de Datos

Ahora, aplicamos las correcciones necesarias para asegurar la integridad y consistencia.

In [ ]:
# 1. Manejo de duplicados
df.drop_duplicates(inplace=True)
print(f"Registros después de eliminar duplicados: {df.shape[0]}")

# 2. Estandarización de texto en 'tipo_vehiculo'
df['tipo_vehiculo'] = df['tipo_vehiculo'].str.strip().str.capitalize()
print("\n--- Conteo de valores después de estandarización ---")
print(df['tipo_vehiculo'].value_counts())

# 3. Manejo de valores nulos
# Para 'duracion_dias', vamos a imputar con la mediana
median_duracion = df['duracion_dias'].median()
df['duracion_dias'].fillna(median_duracion, inplace=True)

# Para 'zona_circulacion', vamos a imputar con el valor más frecuente (moda)
most_frequent_zona = df['zona_circulacion'].mode()[0]
df['zona_circulacion'].fillna(most_frequent_zona, inplace=True)

# 4. Corrección de valores no válidos
# Reemplazar valores negativos en 'duracion_dias' por la mediana
df.loc[df['duracion_dias'] < 0, 'duracion_dias'] = median_duracion

### 3. Verificación Final y Resumen

Finalmente, verificamos que las correcciones hayan sido aplicadas correctamente.

In [ ]:
print("--- Información del DataFrame después de la limpieza ---")
df.info()
print("\n--- Valores faltantes después de la limpieza ---")
print(df.isnull().sum())
print("\n--- Estadísticas descriptivas de 'duracion_dias' limpiada ---")
print(df['duracion_dias'].describe())

# Guardar el dataset limpio
# Crea el directorio 'data/processed' si no existe
import os
os.makedirs('data/processed', exist_ok=True)
df.to_csv('data/processed/permisos_circulacion_limpio.csv', index=False)
print("\nDataset limpio guardado en: data/processed/permisos_circulacion_limpio.csv")